## Diabetic Retinopathy Debrecen Dataset - Decision Tree 

The goal for this analysis is to classify patients as either having or not having diabetic retinopathy. The dataset contains 1151 instances and 20 attributes (categorical and continuous) and can be found [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

### Set up Environment 

In [1]:
# import libraries 
import warnings
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.simplefilter("ignore")

### Read in Data 

In [2]:
# create list w column names 
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('eu_dist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

# read in data, add column names 
data = pd.read_csv("messidor_features.txt", names = col_names)

# preview data 
print(data.info())

data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   quality     1151 non-null   int64  
 1   prescreen   1151 non-null   int64  
 2   ma2         1151 non-null   int64  
 3   ma3         1151 non-null   int64  
 4   ma4         1151 non-null   int64  
 5   ma5         1151 non-null   int64  
 6   ma6         1151 non-null   int64  
 7   ma7         1151 non-null   int64  
 8   exudate8    1151 non-null   float64
 9   exudate9    1151 non-null   float64
 10  exudate10   1151 non-null   float64
 11  exudate11   1151 non-null   float64
 12  exudate12   1151 non-null   float64
 13  exudate13   1151 non-null   float64
 14  exudate14   1151 non-null   float64
 15  exudate15   1151 non-null   float64
 16  eu_dist     1151 non-null   float64
 17  diameter    1151 non-null   float64
 18  amfm_class  1151 non-null   int64  
 19  label       1151 non-null  

,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,eu_dist,diameter,amfm_class,label
350,1,1,44,44,42,32,23,17,172.452792,35.357483,17.933027,5.710940,1.640322,0.574062,0.180098,0.113584,0.528910,0.121771,1,1
697,1,1,56,50,45,39,31,23,105.170765,55.359483,30.940866,7.775036,2.036927,0.865821,0.471709,0.098018,0.568779,0.106186,0,1
638,1,1,6,6,6,6,5,4,47.062703,17.400517,2.614538,0.244024,0.017430,0.000000,0.000000,0.000000,0.529220,0.108683,0,0
194,1,1,74,73,73,66,62,41,51.112703,13.356251,3.772144,0.255739,0.000000,0.000000,0.000000,0.000000,0.547192,0.082202,0,0
628,1,1,24,23,20,18,16,12,128.776696,61.296220,42.974037,9.673157,1.462451,0.296022,0.003116,0.000000,0.485478,0.104906,1,1
22,1,0,37,34,31,30,28,24,8.818234,3.161544,1.900918,1.524727,1.292870,0.165831,0.000000,0.000000,0.538223,0.098270,0,1
174,1,0,13,13,13,12,11,6,31.004416,14.292234,0.994879,0.047469,0.000000,0.000000,0.000000,0.000000,0.549666,0.133508,0,0
118,1,1,9,9,8,7,7,4,145.891998,17.009009,7.400220,0.954294,0.001045,0.000000,0.000000,0.000000,0.532231,0.093025,1,1
1103,1,1,3,3,3,3,3,1,145.210992,74.208340,35.536604,4.212897,0.182014,0.025564,0.004090,0.004090,0.524631,0.106345,1,0
131,1,1,16,15,14,13,11,9,46.832500,21.635597,2.999985,0.506774,0.124958,0.046611,0.000992,0.000992,0.578760,0.101157,1,0


### Data Preprocessing 

Now that the data has been read in, the feature columns need to be separated from the class labels. 

In [3]:
# separate features and labels 
labels = data['label']
features = data.drop(['label'], axis = 1) 

# check shape
print(labels.shape)
print(features.shape)

(1151,)
(1151, 19)


To be able to perform Principal Component Analysis (PCA) later one, the features need to be standardized first. 

In [4]:
# standardize data 
features_scaled = StandardScaler().fit_transform(features) 

# check data 
print('mean =', features_scaled.mean()) # 0 
print('std =', features_scaled.std()) # 1

mean = -4.1263399077018944e-17
std = 1.0


Now, the standardized data needs to be split into training and testing sets for both features and labels with an 80/20 split.

In [5]:
# holdout method; 80/20 split 
lab_train, lab_test, feat_train, feat_test = train_test_split(labels, features_scaled, test_size=0.20)

# check split shapes 
print('lab_train =', lab_train.shape, 'lab_test =', lab_test.shape)
print('feat_train =', feat_train.shape, 'feat_test =', feat_test.shape)

lab_train = (920,) lab_test = (231,)
feat_train = (920, 19) feat_test = (231, 19)


### Principal Component Analysis (PCA) 

Once split, the training set needs to be fit to the model. PCA is affected by the scale of the dataset, which is why the values were standardized previously. Once the data has been fit to the model, the explained variance ration can be extracted to calculate the cumulative sum of it and then determine how many components are needed to explain 95% of the variance.

In [6]:
# create and fit pca 
pca = PCA() 
pca_data = pca.fit_transform(feat_train) 

# explained variance ratio
var_exp = pca.explained_variance_ratio_
print('variance in each pc:', var_exp) 

# calculate cumulative sum of var_exp
cum_var_exp = np.cumsum(var_exp) 
print('\ncumulative variance', cum_var_exp) 

# find index where cum_var_exp reaches 95%
n = 1 + np.argmax(cum_var_exp > 0.95) 
print('\nnumber of columns to keep:', n)

# reduce training set 
feat_train_pca = pca_data[:, :n] 

# transform test set 
feat_test_pca = pca.transform(feat_test)[:, :n]

# check data 
print('\nfeat_train_pca:', feat_train_pca.shape) 
print('feat_test_pca:', feat_test_pca.shape) 

variance in each pc: [3.34651939e-01 2.41657349e-01 1.10726399e-01 6.21819382e-02
 5.30514477e-02 5.05400971e-02 4.29678876e-02 4.04331018e-02
 3.09002565e-02 1.30768655e-02 7.61762625e-03 6.14258087e-03
 2.41088746e-03 1.26111038e-03 1.08181425e-03 8.50622156e-04
 2.62593786e-04 1.21876435e-04 6.36069516e-05]

cumulative variance [0.33465194 0.57630929 0.68703569 0.74921763 0.80226907 0.85280917
 0.89577706 0.93621016 0.96711042 0.98018728 0.98780491 0.99394749
 0.99635838 0.99761949 0.9987013  0.99955192 0.99981452 0.99993639
 1.        ]

number of columns to keep: 9

feat_train_pca: (920, 9)
feat_test_pca: (231, 9)


### Decision Tree

Now that the size of the data has been reduced using PCA, a decision tree classifier can be fit onto the dataset. 

In [7]:
# create decision tree
tree = DecisionTreeClassifier(criterion='entropy')

# build tree 
tree.fit(feat_train_pca, lab_train);

After building the tree with the training data, lets test the accuracy of the model. 

In [8]:
# classify test data 
predict = tree.predict(feat_test_pca) 

# calculate accuracy 
acc = accuracy_score(lab_test, predict) 
print('accuracy:', acc) 

accuracy: 0.5974025974025974


The accuracy of the model came out to 0.57, but this can be improved by modifying the parameters of the model. Let's modify the tree's max depth to see if accuracy can be improved. 

In [9]:
# decision trees w different max_depth
tree_10 = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=4)
tree_15 = DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=4)
tree_20 = DecisionTreeClassifier(criterion='entropy', max_depth=20, random_state=4)

# build trees
tree_10.fit(feat_train_pca, lab_train)
tree_15.fit(feat_train_pca, lab_train)
tree_20.fit(feat_train_pca, lab_train)

# classify test data 
predict_10 = tree_10.predict(feat_test_pca) 
predict_15 = tree_15.predict(feat_test_pca) 
predict_20 = tree_20.predict(feat_test_pca) 

# calculate accuracy 
print('accuracy (max_depth=10):', accuracy_score(lab_test, predict_10))
print('accuracy (max_depth=15):', accuracy_score(lab_test, predict_15))
print('accuracy (max_depth=20):', accuracy_score(lab_test, predict_20))

accuracy (max_depth=10): 0.5584415584415584
accuracy (max_depth=15): 0.6363636363636364
accuracy (max_depth=20): 0.6060606060606061


The holdout method, which is what was used to train and test our data, is not necessarily the best method to calculate accuracy since it can vary depending on the split and the fit of the data. The preferred method, cross-validation, will give a better indication of accuracy since it tests the model with multiple train-test splits. 

### K-fold Cross Validation 

K-fold Cross Validation allows for a better estimate of accuracy. For this analysis, a 10-fold cross validation will be performed. 

In [10]:
# create decision tree 
tree = DecisionTreeClassifier(criterion='entropy')

# get cross validation scores
scores = cross_val_score(tree, features_scaled, labels, cv=10)

# print results
print('scores:', scores) 
print('\naccuracy:', scores.mean())

scores: [0.5862069  0.60869565 0.65217391 0.63478261 0.64347826 0.64347826
 0.62608696 0.57391304 0.67826087 0.6       ]

accuracy: 0.6247076461769114


### Grid Search Cross Validation 

At this point, the accuracy is is still not the best since it would be incorrect about 40% of the time. Next, the model needs to be tuned in order to use the best parameters and avoid overfitting the training data. To do this, lets use Grid Search; This method will evaluate a model for each conbination of algorithm parameters. 

In [11]:
# set up parameters 
params = {'max_depth': [5, 10, 15, 20], 
          'min_samples_leaf': [5, 10, 15, 20], 
          'max_features': [5, 10, 15]
         }

# create gridsearchcv 
grid_search = GridSearchCV(tree, params, cv=5, scoring='accuracy') 

# fit data 
grid_search.fit(features_scaled, labels)

# print results 
print('best parameters:', grid_search.best_params_)
print('\naccuracy:', grid_search.best_score_)

best parameters: {'max_depth': 15, 'max_features': 15, 'min_samples_leaf': 20}

accuracy: 0.6490118577075099


### Nested Cross Validation 

Although the model has improved, it can be taken one step further by running a nested cross validation by passing GridSearchCV into cross_val_score. In doing this, the cross_val_score splits the data into train and test sets for the set number of folds and then calculates the best parameters, score, and estimator for the model. 

In [12]:
# nested cross validation 
nested_score = cross_val_score(grid_search, features_scaled, labels, cv=5, scoring='accuracy')

# print results 
print('accuracy:', nested_score.mean())

accuracy: 0.6263805759457933


The results are not the best given that the model would be wrong 40% of the time. However, to improve our model in the future, some feature engineering could be done on the data. 